In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash_table
import pandas as pd
from collections import OrderedDict
import os
import json
import requests

In [2]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, './style.css'],
                    meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])

In [3]:
pop_model_params = {}
pop_model_params['MobileNet'] = [{"name": "MobileNet-", "description": "Using logistic regression to ", "max_num_models": 100, 
                               "model_selection_algorithm": "GridSearch", "executable_entrypoint": 'Do not need'},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                              
                              {'name': 'Regularization Type', 'param_type': 'hp_choice', 'choices': 'l1,l2', 
                               'min': 'None', 'max': 'None', 'q': 'None', 'dtype': 'dtype_str'},
                             
                              {'name': 'Regularization Strength', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -3, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['MLP'] = [{"name": "MLP-", "description": "Using multilayer perceptron to ", "max_num_models": 100,
                               "model_selection_algorithm": "GridSearch", "executable_entrypoint": 'Do not need'},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                              
                              {'name': 'Regularization Type', 'param_type': 'hp_choice', 'choices': 'l1,l2', 
                               'min': 'None', 'max': 'None', 'q': 'None', 'dtype': 'dtype_str'},
                             
                              {'name': 'Regularization Strength', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -3, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                          
                              {'name': 'Drop Out', 'param_type': 'hp_uniform', 'choices': 'None', 
                               'min': 0.0, 'max': 0.5, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['Bert-Base'] = [{"name": "Bert-Base-", "description": "Using distill BERT to ", "max_num_models": 100,
                               "model_selection_algorithm": "GridSearch", "executable_entrypoint": 'Do not need'},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'}]

pop_model_params['ResNet'] = [{"name": "ResNet-", "description": "Using ResNet to ", "max_num_models": 100,
                               "model_selection_algorithm": "GridSearch", "executable_entrypoint": 'Do not need'},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'},
                             
                              {'name': 'Drop Out', 'param_type': 'hp_uniform', 'choices': 'None', 
                               'min': 0.0, 'max': 0.5, 'q': 'None', 'dtype': 'dtype_float'}]

pop_model_params['Bert-Dil'] = [{"name": "Bert-Distil-", "description": "Using distill BERT to ", "max_num_models": 100,
                               "model_selection_algorithm": "GridSearch", "executable_entrypoint": 'Do not need'},
                              
                              {'name': 'Learning Rate', 'param_type': 'hp_loguniform', 'choices': 'None', 
                               'min': -4, 'max': 1, 'q': 'None', 'dtype': 'dtype_float'},
                              
                              {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
                               'min': 16, 'max': 256, 'q': 16, 'dtype': 'dtype_int'}]


In [4]:
sample_model = [{'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 64, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 64, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 64, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 64, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 96, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 96, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 96, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 96, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 128, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 128, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 128, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 128, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 160, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 160, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 160, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 160, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 192, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 192, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 192, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 192, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 224, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 224, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 224, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 224, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 256, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 256, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 256, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.1, 'batch_size': 256, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 64, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 64, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 64, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 64, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 96, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 96, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 96, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 96, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 128, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 128, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 128, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 128, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 160, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 160, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 160, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 160, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 192, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 192, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 192, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 192, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 224, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 224, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 224, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 224, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 256, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 256, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 256, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.01, 'batch_size': 256, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 64, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 64, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 64, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 64, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 96, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 96, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 96, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 96, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 128, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 128, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 128, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 128, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 160, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 160, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 160, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 160, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 192, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 192, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 192, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 192, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 224, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 224, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 224, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 224, 'dropout': 0.4},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 256, 'dropout': 0.1},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 256, 'dropout': 0.2},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 256, 'dropout': 0.3},
                {'model_name': 'model', 'exp_name': 'ResNet-Example', 'status': 'created', 'lr': 0.001, 'batch_size': 256, 'dropout': 0.4}]

In [5]:
video_model = [{'model_name': 'Amanda',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.0001,   'batch_size': 256, 'model': 'ResNet50'},
               {'model_name': 'Andrew',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.000001, 'batch_size': 32,  'model': 'ResNet50'},
               {'model_name': 'Ashley',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.0001,   'batch_size': 32,  'model': 'VGG16'},
               {'model_name': 'Christopher', 'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.0001,   'batch_size': 256, 'model': 'VGG16'},
               {'model_name': 'Daniel',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.000001, 'batch_size': 32,  'model': 'VGG16'},
               {'model_name': 'David',       'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.000001, 'batch_size': 256, 'model': 'VGG16'},
               {'model_name': 'James',       'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.000001, 'batch_size': 32,  'model': 'ResNet50'},
               {'model_name': 'Jennifer',    'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.0001,   'batch_size': 256, 'model': 'VGG16'},
               {'model_name': 'Jessica',     'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.0001,   'batch_size': 32,  'model': 'ResNet50'},
               {'model_name': 'John',        'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.000001, 'batch_size': 32,  'model': 'VGG16'},
               {'model_name': 'Joseph',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.000001, 'batch_size': 256, 'model': 'VGG16'},
               {'model_name': 'Joshua',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.0001,   'batch_size': 32,  'model': 'ResNet50'},
               {'model_name': 'Matthew',     'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.0001,   'batch_size': 256, 'model': 'ResNet50'},
               {'model_name': 'Michael',     'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.0001,   'batch_size': 32,  'model': 'VGG16'},
               {'model_name': 'Robert',      'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.0001,   'learning_rate': 0.000001, 'batch_size': 256, 'model': 'ResNet50'},
               {'model_name': 'Ryan',        'exp_name': 'ImageNet', 'status': 'running', 'l2_reg': 0.000001, 'learning_rate': 0.000001, 'batch_size': 256, 'model': 'ResNet50'},]
cloned_model = [{'model_name': 'Brandon',    'exp_name': 'ImageNet', 'status': 'created', 'l2_reg': 0.000001, 'learning_rate': 0.00001,  'batch_size': 256, 'model': 'VGG16'},
                {'model_name': 'William',    'exp_name': 'ImageNet', 'status': 'created', 'l2_reg': 0.000001, 'learning_rate': 0.001,    'batch_size': 256, 'model': 'VGG16'}]

In [6]:
with open("model_names.txt", "r") as f:
    model_names = f.read().splitlines()

In [7]:
for ele,name in zip(video_model+cloned_model, model_names):
    ele['parent'] = '/'

In [8]:
for ele in cloned_model:
    ele['parent'] = 'Jennifer'

In [9]:
models = [{'Act': 'None', 'Clone': 'Clone', 'name': 'model 1', 'lr': '1'},
          {'Act': 'None', 'Clone': 'Clone', 'name': 'model 2', 'lr': '0.1'},
          {'Act': 'None', 'Clone': 'Clone', 'name': 'model 3', 'lr': '0.01'},
          {'Act': 'None', 'Clone': 'Clone', 'name': 'model 4', 'lr': '0.001'}]

In [10]:
EXPERIMENT_ELEMENT = (
    "name", "description", "max_num_models", "feature_columns", "label_columns", 
    "max_train_epochs", "data_store_prefix_path", "executable_entrypoint", "model_selection_algorithm"
)
EXPERIMENT_TYPE = (
    "string", "string", "int", "string", "string", "int", "string", "string", "string"
)
exp_describe_id = (
    "exp_describe_name", 
    "exp_describe_describe", 
    "exp_describe_max_model",
    "exp_describe_features", 
    "exp_describe_label", 
    "exp_describe_max_epoch",
    "exp_describe_data_path", 
    "exp_describe_func",
    "exp_describe_search_strategy"
)
describe_list = (
    "Name of experiment (string type)", 
    "Description (optional)", 
    "Max number of models for random and autoML (integer type)",
    "Name of features (string type, comma seperated)", 
    "Name of label (string type)", 
    "Maximum training epochs (integer type)",
    "Data store prefix path (string type)", 
    "Estimator function name (<module_name>:<function_name>)",
    "Hyperparameters search strategy (please select)"
)

In [11]:
tensorboard_frame = html.Iframe(src="http://localhost:6006/", style={"height": "1067px", "width": "100%"})

In [12]:
# App layout
app.layout = dbc.Container([

    dbc.Navbar(
        [
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [dbc.Col(dbc.NavbarBrand("Intermittent Human-in-the-Loop Cerebro", className="ml-2", style = {'font-size':'xx-large'}))],
                align="center",
                no_gutters=True,
            )
        ],
        color='#FF6F00',
        dark=True, 
        style = {'width': '100%'}
    ),
    
#     dbc.Row( dbc.Col(html.H1("Human In Loop Cerebro Training", className='text-center'))),

#     html.Hr(),
    
    # *************************************************************************************************
    #
    #     Model selection section
    #
    # *************************************************************************************************
    
    dbc.Row([dbc.Col(html.H3("Upload a model script file OR choose a popular model", className='text-center',
                             style={"margin-top": "15px"}))]),
    
    dbc.Row( [
        dbc.Col(
            dcc.Upload(
                id='upload-script',
                children = ['Drag and Drop or ',
                    html.A('Select a File', style ={ 'text-decoration': 'underline'})
                ], 
                style={
                    'width': '100%',
                    'height': '130px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center'
                },
                multiple=True
            ),
            className='column_left'
        ), 
        dbc.Col([
            html.H6("Select a popular model:"),
            dcc.Dropdown(id="slct_pop_model",
                 options=[
                     {"label": "MobileNet", "value": "MobileNet"},
                     {"label": "MLP", "value": "MLP"},
                     {"label": "BERT-base", "value": "Bert-Base"},
                     {"label": "ResNet-50", "value": "ResNet"},
                     {"label": "DistillBERT", "value": "Bert-Dil"}],
                 multi=False,
                 value="ResNet"
            ),
            html.Button('Select this model', id='slct_model', n_clicks=0, style={"margin-top": "15px"})
        ])
    ]),
    
    html.Div(id='script_response', children=[]),
    
    html.Hr(),
    
    # *************************************************************************************************
    #
    #     Experiment setup section
    #
    # *************************************************************************************************
    
    html.H3("Setup experiment"),
    
    html.Div(id='experiment-part', children=[
        dbc.Row([
            dbc.Col(
                [
                    dbc.Row([html.Div(id=Id, children=Describe, style= {'display': 'block'})]) for Id, Describe in zip(exp_describe_id, describe_list)
                ], width=8
            ),
            dbc.Col(
                [ 
                    dbc.Row([dcc.Input(
                        id="exp_{}".format(e),
                        value=None,
                        type=t,
                        placeholder="{}:({})".format(e,t),
                        style={
                        'width': '240px',
                        'height': '25px'}
                    )])
                    for e,t in zip(EXPERIMENT_ELEMENT[:-1],EXPERIMENT_TYPE[:-1])
                ] +
                [
                    dbc.Row([dcc.Dropdown(id="model_slct_algorithm",
                         options=[
                             {"label": "GridSearch", "value": "GridSearch"},
                             {"label": "RandomSearch", "value": "RandomSearch"},
                             {"label": "HyperOpt", "value": "HyperOpt"}],
                         multi=False,
                         value="RandomSearch",
                         style={'width': "90%", 'height': '35px', 'display': 'block'}
                     )])
                ], width=4
            )
        ]),

        html.H5("Hyperparameter grid table:", style={"margin-top": "15px"}),

        dash_table.DataTable(
            id='datatable-params',
            data=[], #pop_model_params['ResNet'][1:],
            columns=[
                {"name": 'Name', 'id': 'name', "selectable": True},
                {"name": "Hyperparameter Type", "id": "param_type", 'presentation': 'dropdown'},
                {"name": "Choices", "id": "choices", "selectable": True},
                {"name": "Min Value", "id": "min", "selectable": True},
                {"name": "Max Value", "id": "max", "selectable": True},
                {"name": "Quantum", "id": "q", "selectable": True},
                {"name": "Data Type", "id": "dtype", 'presentation': 'dropdown'}
            ],
            editable=True,
            row_deletable=True,
            filter_action="native",     # allow filtering of data by user ('native') or not ('none')
            page_current=0,             # page number that user is on
            page_size=20,                # number of rows visible per page
            style_cell={                # ensure adequate header width when text is shorter than cell's text
                'width': 'auto', 'height': 'auto', #'whiteSpace': 'normal'
            },
            style_cell_conditional=[    # align text columns to left. By default they are aligned to right
                {
                    'if': {'column_id': c},
                    'textAlign': 'left'
                } for c in ['name', 'param_type', "choices", "min", "max", "q", "dtype"]
            ],
            style_data={                # overflow cells' content into multiple lines
                'whiteSpace': 'normal',
                'height': 'auto'
            },
            dropdown={
                'param_type': {
                    'options': [
                        {'label': 'categorical', 'value': 'hp_choice'},
                        {'label': 'uniform', 'value': 'hp_uniform'},
                        {'label': 'quantum uniform', 'value': 'hp_quniform'},
                        {'label': 'log uniform (base 10)', 'value': 'hp_loguniform'},
                        {'label': 'quantum log uniform (base 10)', 'value': 'hp_qloguniform'}
                    ],
                    'value':'hp_choice'
                },
                'dtype': {
                    'options': [
                        {'label': 'string', 'value': 'dtype_str'},
                        {'label': 'integer', 'value': 'dtype_int'},
                        {'label': 'float', 'value': 'dtype_float'}
                    ],
                    'value':'dtype_str'
                }
            }
        ),    

        html.Button('Add New Parameter(One row)', id='add_param', n_clicks=0, 
                    style={"margin-bottom": "15px", "margin-top": "15px"}),

        html.H5("Please check info above and launch the experiment by clicking the button below:"),
        html.Button('Launch This Experiment', id='add_exp', n_clicks=0, 
                        style = {'height': '40px', 'textAlign': 'center',
                                 'color': 'white', 'background-color': '#FF6F00', 'font-weight': 'bold'}),      
        html.Div(id='param_output', children=[])], style= {'display': 'none'}),
    html.Hr(),
 
    # *************************************************************************************************
    #
    #     Models inspection section
    #
    # *************************************************************************************************
    
    html.H3("Models"),
     
    html.Div(id="model_part", children=[
        html.H6("Input model name and operate on the model:"),
        dcc.Input(id="model_name_to_change", type="string", placeholder="Model Name",
                        style={'width': '200px', 'height': '35px', "margin-bottom": "5px"}),
        html.Br(), 
        dbc.Row([html.Button('Resume', id='resume_', n_clicks=0, style={"margin-left": "5px", "margin-left": "2px"}),
                             html.Button('Delete', id='delete_', n_clicks=0, style={"margin-left": "2px", "margin-left": "2px"}),
                             html.Button('Stop', id='stop_', n_clicks=0, style={"margin-left": "2px", "margin-left": "2px"}),
                             html.Button('Clone', id='clone_', n_clicks=0, style={"margin-left": "2px", "margin-left": "2px"})],
                 style = {"margin-bottom": "15px"}),

        dbc.Modal(
            [
                dbc.ModalHeader("Clone Model"),
                dbc.ModalBody(id='modal_body', children=[]),
                dbc.ModalFooter(
                    dbc.Button("Confirm", id="close_clone", n_clicks=0, className="ml-auto", 
                               style = {'color': 'white', 'background-color': '#FF6F00'})
                ),
            ],
            id="clone_modal",
            centered=True,
            is_open=False,
            style={"max-width": "none", "width": "90%"}
        ),

        dash_table.DataTable(
            id='datatable-interactivity',
            data=video_model,
            columns=[
                {"name": "Model Name", "id": "model_name", "selectable": True },
                {"name": "Experiment Name", "id": "exp_name", "selectable": True},
                {"name": "Parent", "id": "parent", "selectable": False},
                {"name": "Status", "id": "status", "selectable": False},
                {"name": "Learning Rate", "id": "learning_rate", "selectable": True},
                {"name": 'Batch Size', 'id': 'batch_size', "selectable": True},
                {"name": 'Model', 'id': 'model', "selectable": True},
                {"name": 'l2_Regularization', 'id': 'l2_reg', "selectable": True},
            ],
            editable=True,
            sort_action='native',
            filter_action="native",     # allow filtering of data by user ('native') or not ('none')
            page_current=0,             # page number that user is on
            page_size=100,                # number of rows visible per page
            style_cell={                # ensure adequate header width when text is shorter than cell's text
                'minWidth': 60, 'maxWidth': 120, 'width': 80
            },
            style_cell_conditional=[    # align text columns to left. By default they are aligned to right
                {
                    'if': {'column_id': c},
                    'textAlign': 'left'
                } for c in ["model_name", "exp_name", "parent","status"]
            ],
            style_data={                # overflow cells' content into multiple lines
                'whiteSpace': 'normal',
                'height': 'auto'
            }
        )], style= {'display': 'none'}),
    
    html.Hr(),
    html.Div(id='tensorboard', children=[tensorboard_frame]),
    html.Hr(),

])

In [13]:
jennifer = [{'name': 'learning_rate', 'param_type': 'hp_qloguniform', 'choices': 'None', 
            'min': -3, 'max': -1, 'q': 2, 'dtype': 'dtype_float'}]
# michael = [{'name': 'Learning Rate', 'param_type': 'hp_qloguniform', 'choices': 'None', 
#             'min': -3, 'max': -1, 'q': 0.5, 'dtype': 'dtype_float'},

#            {'name': 'Batch Size', 'param_type': 'hp_quniform', 'choices': 'None', 
#             'min': 64, 'max': 64, 'q': 0, 'dtype': 'dtype_int'},

#            {'name': 'Drop Out', 'param_type': 'hp_uniform', 'choices': 'None', 
#             'min': 0.1, 'max': 0.1, 'q': 'None', 'dtype': 'dtype_float'}]

In [14]:
clone_checkbox = dcc.RadioItems(
                    options=[
                        {'label': 'Warm Start on Base Model', 'value': 'warm'},
                        {'label': 'Start From Scratch', 'value': 'new'}
                    ],
                    value='warm',
                    labelStyle={'display': 'inline-block', "margin-right": "20px"},
                    style={"margin-bottom": "5px"}
                )  
clone_table = dash_table.DataTable(
                id='clone_table',
                data=jennifer,
                columns=[
                    {"name": 'Name', 'id': 'name', "selectable": True},
                    {"name": "Hyperparameter Type", "id": "param_type", 'presentation': 'dropdown'},
                    {"name": "Choices", "id": "choices", "selectable": True},
                    {"name": "Min Value", "id": "min", "selectable": True},
                    {"name": "Max Value", "id": "max", "selectable": True},
                    {"name": "Quantum", "id": "q", "selectable": True},
                    {"name": "Data Type", "id": "dtype", 'presentation': 'dropdown'}
                ],
                editable=True,
                page_current=0,             # page number that user is on
                page_size=20,                # number of rows visible per page
                style_cell={                # ensure adequate header width when text is shorter than cell's text
                    'width': 'auto', 'height': 'auto', #'whiteSpace': 'normal'
                },
                style_cell_conditional=[    # align text columns to left. By default they are aligned to right
                    {
                        'if': {'column_id': c},
                        'textAlign': 'left'
                    } for c in ['name', 'param_type', "choices", "min", "max", "q", "dtype"]
                ],
                style_data={                # overflow cells' content into multiple lines
                    'whiteSpace': 'normal',
                    'height': 'auto'
                },
                dropdown={
                    'param_type': {
                        'options': [
                            {'label': 'categorical', 'value': 'hp_choice'},
                            {'label': 'uniform', 'value': 'hp_uniform'},
                            {'label': 'quantum uniform', 'value': 'hp_quniform'},
                            {'label': 'log uniform', 'value': 'hp_loguniform'},
                            {'label': 'quantum log uniform', 'value': 'hp_qloguniform'}
                        ],
                        'value':'hp_choice'
                    },
                    'dtype': {
                        'options': [
                            {'label': 'string', 'value': 'dtype_str'},
                            {'label': 'integer', 'value': 'dtype_int'},
                            {'label': 'float', 'value': 'dtype_float'}
                        ],
                        'value':'dtype_str'
                    }
                }
            )   

In [15]:
clone_name_form = dbc.FormGroup(
    [
        dbc.Label("Base Model Name", html_for="clone-name-row", width=3),
        dbc.Col(
            dbc.Input(
                type="text", id="clone-name-row", placeholder="Enter name", value = "Jennifer"
            ),
            width=9,
        ),
    ],
    row=True,
)
clone_lr_form = dbc.FormGroup(
    [
        dbc.Label("Learning Rate", html_for="clone-lr-row", width=3),
        dbc.Col(
            dbc.Input(
                type="number", id="clone-lr-row", placeholder="Enter learning rate", value = 0.1
            ),
            width=9,
        ),
    ],
    row=True,
)
clone_batch_form = dbc.FormGroup(
    [
        dbc.Label("Batch Size", html_for="clone-batch-row", width=3),
        dbc.Col(
            dbc.Input(
                type="number", id="clone-batch-row", placeholder="Enter batch size", value = 64
            ),
            width=9,
        ),
    ],
    row=True,
)
clone_dropout_form = dbc.FormGroup(
    [
        dbc.Label("Dropout", html_for="clone-dropout-row", width=3),
        dbc.Col(
            dbc.Input(
                type="number", id="clone-dropout-row", placeholder="Enter dropout", value = 0.1
            ),
            width=9,
        ),
    ],
    row=True,
)

In [16]:
@app.callback([Output('upload-script', 'contents'),
              Output('script_response', 'children'),
              Output('experiment-part', 'style')],
              Input('upload-script', 'contents'),
              State('upload-script', 'filename'))
def upload_script(list_of_contents, list_of_names):
    printout = None
    style= {'display': 'none'}
    if list_of_contents is not None:
        print("contents:", list_of_contents)
        print("filename:", list_of_names)
#         script_post_url = "http://localhost:8889/api/scripts/upload"
#         r = requests.post(script_post_url, files={'file': list_of_contents[0]})
#         if r.status_code==201:
#             print("A script has been successfully uploaded!")
#             printout = "A script has been successfully uploaded!"
        style['display'] = 'block'
    return None, printout, style

In [17]:
@app.callback(
    [Output(component_id='datatable-params', component_property='data'),
     Output(component_id='add_param', component_property='n_clicks'), 
     Output(component_id='slct_model', component_property='n_clicks'),
     Output(component_id='datatable-params', component_property='editable'),
     Output(component_id='exp_name', component_property='value'),
     Output(component_id='exp_description', component_property='value'),
     Output(component_id='exp_max_num_models', component_property='value'),
     Output(component_id='exp_feature_columns', component_property='value'),
     Output(component_id='exp_label_columns', component_property='value'),
     Output(component_id='exp_max_train_epochs', component_property='value'),
     Output(component_id='exp_data_store_prefix_path', component_property='value'),
     Output(component_id='exp_executable_entrypoint', component_property='value')],
    [Input(component_id='add_param', component_property='n_clicks'),
     Input(component_id='slct_model', component_property='n_clicks')],
    [State('datatable-params', 'data'), State('datatable-params', 'editable'), State('slct_pop_model', 'value')]
)
def add_param(add_param_n_clicks, slct_model_n_click, data, editable, value):
    print('Button has been clicked.')   
    
    exp_setup_list = [None, None, None, None, None, None, None, None]
    
    if add_param_n_clicks > 0:
        data.append({'param_name': 'name?', 'param_type': 'categorical', 'param_values': 'None', 'param_min_val': 0, 
                      'param_max_val': 0, 'param_count': 0, 'param_base': 0})
    
    if slct_model_n_click > 0:
        data = pop_model_params[value][1:]
        editable = False
        exp_setup_list[0] = pop_model_params[value][0]['name']
        exp_setup_list[1] = pop_model_params[value][0]['description']
        exp_setup_list[2] = pop_model_params[value][0]['max_num_models']
        exp_setup_list[7] = pop_model_params[value][0]['executable_entrypoint']
        
    return [data, 0, 0, editable] + exp_setup_list

In [18]:
@app.callback(
    [Output(component_id='datatable-interactivity', component_property='data'),
     Output(component_id='datatable-interactivity', component_property='columns'),
     Output(component_id='add_exp', component_property='n_clicks'),
     Output('modal_body', 'children'),
     Output('clone_modal', 'is_open'),
     Output('clone_', 'n_clicks'),
     Output('close_clone', 'n_clicks'),
     Output('resume_', 'n_clicks'), 
     Output('delete_', 'n_clicks'),
     Output('stop_', 'n_clicks'),
     Output('model_part', 'style'),
     Output('exp_describe_max_model', 'style'),
     Output('exp_max_num_models', 'style')],
    [Input('model_slct_algorithm', 'value'),
     Input(component_id='add_exp', component_property='n_clicks'), 
     Input('clone_', 'n_clicks'), Input('close_clone', 'n_clicks'),
     Input('resume_', 'n_clicks'), Input('delete_', 'n_clicks'),
     Input('stop_', 'n_clicks')],
    [State('datatable-params', 'data'), State(component_id='datatable-interactivity', component_property='data'),
     State(component_id='datatable-interactivity', component_property='columns'),
     State('clone_modal', 'is_open'), State('model_name_to_change', 'value'),
     State('tensorboard', 'children'), State('exp_describe_max_model', 'style'),
     State('exp_max_num_models', 'style')]
)
def add_exp(model_slct_algorithm, n_clicks_add_exp, clone_n_clicks, close_clone_n_clicks, resume_n_clicks,
            delete_n_clicks, stop_n_clicks, data, return_data, columns, clone_modal_is_open, 
            model_name_to_change, tensorboard, exp_describe_max_model_style, exp_max_num_models_style):
    if model_slct_algorithm == "GridSearch":
        exp_describe_max_model_style['display'] = 'none'
        exp_max_num_models_style['display'] = 'none'
    else:
        exp_describe_max_model_style['display'] = 'block'
        exp_max_num_models_style['display'] = 'block'
    
    if n_clicks_add_exp > 0:
#         exp_post_url = "http://localhost:8889/api/experiments/"
#         r = requests.post(exp_post_url, json=payload)
#         print(r)
#         print(r.json())
        return_data = video_model

        columns=[
                {"name": "Model Name", "id": "model_name", "selectable": True },
                {"name": "Experiment Name", "id": "exp_name", "selectable": True},
                {"name": "Parent", "id": "parent", "selectable": False},
                {"name": "Status", "id": "status", "selectable": False},
                {"name": "Learning Rate", "id": "learning_rate", "selectable": True},
                {"name": 'Batch Size', 'id': 'batch_size', "selectable": True},
                {"name": 'Model', 'id': 'model', "selectable": True},
                {"name": 'l2_Regularization', 'id': 'l2_reg', "selectable": True}
        ]
        
        tensorboard.append(tensorboard_frame)
        
        return return_data, columns, 0, None, clone_modal_is_open, 0, 0, 0, 0, 0, {'display': 'block'}, \
                exp_describe_max_model_style, exp_max_num_models_style

    if clone_n_clicks > 0:
        return return_data, columns, 0, [clone_name_form, clone_checkbox, clone_table],\
                not clone_modal_is_open, 0, 0, 0, 0, 0, {'display': 'block'}, exp_describe_max_model_style, exp_max_num_models_style
    
    if close_clone_n_clicks > 0:
        return return_data+cloned_model, columns, 0, None, not clone_modal_is_open, \
                0, 0, 0, 0, 0, {'display': 'block'}, exp_describe_max_model_style, exp_max_num_models_style
        
    if resume_n_clicks > 0:
        if model_name_to_change is not None:
            for model in return_data:
                if model['model_name'] == model_name_to_change:
                    model['status'] = 'running'
        return return_data, columns, 0, None, clone_modal_is_open, 0, 0, 0, 0, 0, {'display': 'block'}, \
                exp_describe_max_model_style, exp_max_num_models_style
        
    if delete_n_clicks > 0:
        if model_name_to_change is not None:
            for model in return_data:
                if model['model_name'] == model_name_to_change:
                    return_data.remove(model)
        return return_data, columns, 0, None, clone_modal_is_open, 0, 0, 0, 0, 0, {'display': 'block'}, \
                exp_describe_max_model_style, exp_max_num_models_style
        
    if stop_n_clicks > 0:
        if model_name_to_change is not None:
            for model in return_data:
                if model['model_name'] == model_name_to_change:
                    model['status'] = 'stopped'
        return return_data, columns, 0, None, clone_modal_is_open, 0, 0, 0, 0, 0, {'display': 'block'}, \
                exp_describe_max_model_style, exp_max_num_models_style
    
    return return_data, columns, 0, None, clone_modal_is_open, 0, 0, 0, 0, 0, {'display': 'none'}, \
            exp_describe_max_model_style, exp_max_num_models_style

In [19]:
payload = {
  "name": "ImageNet",
  "description": "",
  "model_selection_algorithm": "GridSearch",
  "param_defs": [
    {
      "name": "learning_rate",
      "param_type": "hp_qloguniform",
      "min": -6,
      "max": -4,
      "q": 2,
      "dtype": "dtype_float"
    },
    {
      "name": "batch_size",
      "param_type": "hp_choice",
      "choices": "32,256",
      "dtype": "dtype_int"
    },
    {
      "name": "l2_reg",
      "param_type": "hp_qloguniform",
      "min": -6,
      "max": -4,
      "q": 2,
      "dtype": "dtype_float"
    },
    {
      "name": "model",
      "param_type": "hp_choice",
      "choices": "ResNet50,VGG16",
      "dtype": "dtype_str"
    }
  ],
  "feature_columns": "image",
  "label_columns": "label",
  "max_train_epochs": 10,
  "data_store_prefix_path": "hdfs://storage_server/imagenet",
  "executable_entrypoint": "ImageNet_model_gen_script:estimator_gen_fn"
}

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False, port=8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2021 16:05:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:27] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:27] "GET /assets/style.css?m=1614702173.316204 HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:27] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_0m1613478971.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:28] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_0m1613478971.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:28] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_0m1613478971.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:28] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_15_0m1613478972.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:05:28] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_0m1613478971.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - -

Button has been clicked.


127.0.0.1 - - [03/May/2021 16:06:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:06:17] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.


...

127.0.0.1 - - [03/May/2021 16:11:54] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.


127.0.0.1 - - [03/May/2021 16:11:55] "POST /_dash-update-component HTTP/1.1" 200 -


Button has been clicked.


127.0.0.1 - - [03/May/2021 16:14:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:39:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:40:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:46:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:46:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:46:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:47:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:47:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 01:36:18] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 01:36:19] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -


# Break point here

In [ ]:
[{'props': {
    'children': [
        {'props': {
             'children': {'props': {'id': 'param_name_lr', 'type': 'string', 
             'placeholder': 'lr'}, 'type': 'Input', 'namespace': 'dash_core_components'}
         }, 
         'type': 'Col', 
         'namespace': 'dash_bootstrap_components'}
    ]
  }, 
  'type': 'Row', 
  'namespace': 'dash_bootstrap_components'}, 
 
 {'props': {'children': [{'props': {'children': {'props': {'id': 'param_name_penalty', 'type': 'string', 
    'placeholder': 'penalty'}, 'type': 'Input', 'namespace': 'dash_core_components'}}, 'type': 'Col', 
    'namespace': 'dash_bootstrap_components'}]}, 'type': 'Row', 'namespace': 'dash_bootstrap_components'}, 
 {'props': {'children': [{'props': {'children': {'props': {'id': 'param_name_C', 'type': 'string', 'placeholder': 'C'},
    'type': 'Input', 'namespace': 'dash_core_components'}}, 'type': 'Col', 'namespace': 'dash_bootstrap_components'}]},
  'type': 'Row', 'namespace': 'dash_bootstrap_components'}]

## Try requests

In [15]:
script_post_url = "http://localhost:8889/api/scripts/upload"
with open('model_gen_script.py', 'rb') as f:
    r = requests.post(script_post_url, files={'file': f})

In [17]:
r

<Response [201]>

## Original Sample Page

In [12]:
import dash
import dash_html_components as html
import dash_table
import pandas as pd
from collections import OrderedDict


app = dash.Dash(__name__)

df = pd.DataFrame(OrderedDict([
    ('climate', ['Sunny', 'Snowy', 'Sunny', 'Rainy']),
    ('temperature', [13, 43, 50, 30]),
    ('city', ['NYC', 'Montreal', 'Miami', 'NYC'])
]))


app.layout = html.Div([
    dash_table.DataTable(
        id='table-dropdown',
        data=df.to_dict('records'),
        columns=[
            {'id': 'climate', 'name': 'climate', 'presentation': 'dropdown'},
            {'id': 'temperature', 'name': 'temperature'},
            {'id': 'city', 'name': 'city', 'presentation': 'dropdown'},
        ],

        editable=True,
        dropdown={
            'climate': {
                'options': [
                    {'label': i, 'value': i}
                    for i in df['climate'].unique()
                ]
            },
            'city': {
                 'options': [
                    {'label': i, 'value': i}
                    for i in df['city'].unique()
                ]
            }
        }
    ),
    html.Div(id='table-dropdown-container')
])


if __name__ == '__main__':
    app.run_server(debug=False, port=8051)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 21:58:02] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -


## Sample Data frame

In [3]:
df = pd.DataFrame(OrderedDict([
    ('Action', ['Sunny', 'Snowy']),
    ('Model Name', ['Model 1','Model 2']),
    ('Parameter', ['0.1', '0.01'])
]))

In [4]:
df.to_dict('records')

[{'Action': 'Sunny', 'Model Name': 'Model 1', 'Parameter': '0.1'},
 {'Action': 'Snowy', 'Model Name': 'Model 2', 'Parameter': '0.01'}]

In [ ]:
@app.callback([Output('modal_body', 'children'), Output('clone_modal', 'is_open'), Output('clone_', 'n_clicks'), 
               Output('close_clone', 'n_clicks')],
              [Input('clone_', 'n_clicks'), Input('close_clone', 'n_clicks')],
              [State('clone_modal', 'is_open'), State('model_name_to_change', 'value')]
)
def clone_function(clone_n_clicks, close_clone_n_clicks, is_open, name_to_change):
    print("Clone button clicked!")
    if clone_n_clicks > 0:
        return [clone_name_form, clone_checkbox, clone_table], not is_open, 0, 0
    if close_clone_n_clicks > 0:
        return None, not is_open, 0, 0
    
    return None, False, 0, 0

In [ ]:
@app.callback(
    Output(component_id='datatable-interactivity', component_property='data'),
    [Input(component_id='add_model', component_property='n_clicks')],
    [State('datatable-interactivity', 'data')]
)
def add_model(n_clicks, data):
    print('Button has been clicked.')
    
    if n_clicks > 0:
        data.append({'Act': 'Add', 'Name': f'Model {len(data)+1}', 'Param': 'input later'})

    return data